In [1]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
import json

_ = load_dotenv(find_dotenv())

client = OpenAI()

### Steps

#### 1. Define `Functions`
#### 2. Create an `Assitant` and register Functions
#### 3. Create a `Thread`
#### 4. Add a `Message` to the Thread
#### 5. `Run` the Assistant
#### 6. Polling for updates and `Calling` Functions

## 1. Define Functions

In [2]:
# Define the functions


def getCurrentWeather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location):
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

## 2. Create an Assistant and register functions to it

In [3]:
assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo-1106",
    #  register functions in tools
    tools=[{
        "type": "function",
        "function": {
            # first function
            "name": "getCurrentWeather",
            "description": "Get the weather in location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
                    "unit": {"type": "string", "enum": ["c", "f"]}
                },
                "required": ["location"]
            }
        }
    }, {
        "type": "function",
        "function": {
            # second function
            "name": "getNickname",
            "description": "Get the nickname of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
                },
                "required": ["location"]
            }
        }
    }]
)

dict(assistant)


{'id': 'asst_VzaYlPg7WfcJ5Gu8xMwK5pOc',
 'created_at': 1703702267,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['location']}, description='Get the nickname of a city'), type='function')]}

## 3. Create a Thread

In [4]:
thread = client.beta.threads.create()

print(thread)

Thread(id='thread_x67K9JO3l3AUbFkEbno9KrJ6', created_at=1703702268, metadata={}, object='thread')


## 4. Add a Message

In [6]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What is the temperature in Karachi"
)

dict(message)

{'id': 'msg_tr8FlYXUZGsuHC7lkIcpvhWc',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='What is the temperature in Karachi'), type='text')],
 'created_at': 1703702317,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_x67K9JO3l3AUbFkEbno9KrJ6'}

## 5. Run the Assistant

In [13]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,    
)

dict(run)

{'id': 'run_Ojrt3mD11oOswzgT4uWDZHqx',
 'assistant_id': 'asst_VzaYlPg7WfcJ5Gu8xMwK5pOc',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703702666,
 'expires_at': 1703703266,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_x67K9JO3l3AUbFkEbno9KrJ6',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object